## Cosmological Emulator

In this example, we will calculate the matter power spectrum using Cosmological Emulator such as MiraTitan and BaccoEmu.

In [ ]:
import pyccl as ccl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.rcParams['figure.figsize']=(6.5,5.5)
np.set_printoptions(linewidth=120)
np.get_printoptions()

### baccoemu

These below sentences are based on https://baccoemu.readthedocs.io/en/latest/#

We begin the range of wavenumber and redshift.
you can check the k-range on which the linear/non-linear emulator is defined you can type

In [ ]:
import baccoemu
emulator = baccoemu.Matter_powerspectrum()
k_range_lin = emulator.emulator['linear']['k']
k_range_nl = emulator.emulator['nonlinear']['k']

In [ ]:
print("linear:", "min:", min(k_range_lin), "[h/Mpc]", "max:", max(k_range_lin), "[h/Mpc]")
print("non-linear:", "min:", min(k_range_nl), "[h/Mpc]", "max:", max(k_range_nl), "[h/Mpc]")

In [ ]:
kmin, kmax, nk = min(k_range_nl), max(k_range_nl), 128
k_bemu = np.logspace(np.log10(kmin*0.67), np.log10(kmax*0.67), nk) # Wavenumber [Mpc]^-1
a = 1. # Scale factor a z=0

In [ ]:
bemu_lin = ccl.BaccoemuLinear()
cosmo_lin = ccl.Cosmology(Omega_c=0.27, Omega_b=0.05, h=0.67, n_s=0.96, sigma8=0.83,
                      m_nu=0.1, transfer_function='boltzmann_camb',
                      matter_power_spectrum=bemu_lin)

In [ ]:
bemu_nl = ccl.BaccoemuNonlinear()
cosmo_nl = ccl.Cosmology(Omega_c=0.27, Omega_b=0.05, h=0.67, n_s=0.96, sigma8=0.83,
                      m_nu=0.1, transfer_function='boltzmann_camb',
                      matter_power_spectrum=bemu_nl)

In [ ]:
# Plot linear and nonlinear power spectra
pk_lin_bemu = cosmo_lin.get_linear_power()
pk_nl_bemu = cosmo_nl.get_nonlin_power()

In [ ]:
plt.plot(k_bemu, pk_lin_bemu(k_bemu, a), 'b-', label="linear")
plt.plot(k_bemu, pk_nl_bemu(k_bemu, a), 'r-', label="non-linear")

plt.xscale('log')
plt.yscale('log')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='lower left',fontsize=10)
plt.xlabel('$k\quad[\\rm Mpc^{-1}]$', fontsize=22)
plt.ylabel('$P(k)\quad[{\\rm Mpc}]^3$', fontsize=22)
plt.title("BaccoEmu", fontsize=22)

plt.tight_layout()
plt.show()

### baccoemu baryon-corrected matter power spectrum emulator

You can also compute the baryon corrected power spectrum emulator.

In [ ]:
baryons = ccl.BaccoemuBaryons()

pk2d_gro = cosmo_nl.get_nonlin_power()
pk2d_bcm = baryons.include_baryonic_effects(cosmo_nl, pk2d_gro)

In [ ]:
plt.plot(k_bemu, pk2d_bcm(k_bemu, a) / pk2d_gro(k_bemu, a))

plt.xscale('log')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('$k\quad[\\rm Mpc^{-1}]$', fontsize=22)
plt.ylabel('$P_{\\rm baryon}(k)/P_{\\rm nl}(k)$', fontsize=22)

plt.tight_layout()
plt.show()

If you want to update the parameters, you can type

In [ ]:
baryons.update_parameters(log10_M_c=12.7)

### MiraTitan

In [ ]:
kmin, kmax, nk = 1e-4, 1e1, 128
k_cemu = np.logspace(np.log10(kmin), np.log10(kmax), nk) # Wavenumber [Mpc]^-1
a = 1. # Scale factor a z=0

In [ ]:
cemu = ccl.CosmicemuMTIVPk("tot")
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.05, h=0.67, sigma8=0.8, n_s=0.96, 
                      Neff=3.04, transfer_function='boltzmann_class', matter_power_spectrum=cemu)

Notice we have one class for the 2017 emulator (which is the one CCL was using up to version 2). That's the MiraTitan II (hence the name MTII). Another class is available for the 2022 emulator (MTIV). For each of them there are emulators for the total matter power spectrum 'tot' (i.e. including neutrinos) or for CDM+baryons 'cb'. So you can choose amongst all these options:

In [ ]:
#cemu = ccl.CosmicemuMTIIPk("tot")
#cemu = ccl.CosmicemuMTIIPk("cb")
#cemu = ccl.CosmicemuMTIVPk("tot")
#cemu = ccl.CosmicemuMTIVPk("cb")

Then we proceed as usual for obtaining the matter power spectrum.

In [ ]:
# Plot linear and nonlinear power spectra
pk_lin_cemu = ccl.linear_matter_power(cosmo, k_cemu, a)
pk_nl_cemu = ccl.nonlin_matter_power(cosmo, k_cemu, a)

In [ ]:
plt.plot(k_cemu, pk_lin_cemu, 'b-', label="linear")
plt.plot(k_cemu, pk_nl_cemu, 'r-', label="non-linear")

plt.xscale('log')
plt.yscale('log')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='lower left',fontsize=10)
plt.xlabel('$k\quad[\\rm Mpc^{-1}]$', fontsize=22)
plt.ylabel('$P(k)\quad[{\\rm Mpc}]^3$', fontsize=22)
plt.title("MiraTitan IV", fontsize=22)

plt.tight_layout()
plt.show()